### Open namespace and get extensions for multichannel volumetric data

In [1]:
import os
from pynwb import load_namespaces, get_class
from pynwb.file import MultiContainerInterface, NWBContainer

# Set path of the namespace.yaml file to the expected install location
MultiChannelVol_specpath = os.path.join(
    os.getcwd(),
    'spec',
    'ndx-multichannel-volume.namespace.yaml'
)
# Load the namespace
load_namespaces(MultiChannelVol_specpath)

# TODO: import your classes here or define your class using get_class to make
# them accessible at the package level
#MultiChannelVolume = get_class('MultiChannelVolume', 'ndx-multichannel-volume')
#ImagingVolume = get_class('ImagingVolume', 'ndx-multichannel-volume')
OpticalChannelReferences = get_class('OpticalChannelReferences', 'ndx-multichannel-volume')
#VolumeSegmentation = get_class('VolumeSegmentation', 'ndx-multichannel-volume')


### Define VolumeSegmentation class and add necessary functions

In [2]:
from collections.abc import Iterable
import numpy as np
from pynwb import register_class
from hdmf.utils import docval, get_docval, popargs
from pynwb.ophys import ImageSeries 
from pynwb.core import NWBDataInterface
from hdmf.common import DynamicTable
from hdmf.utils import docval, popargs, get_docval, get_data_shape, popargs_to_dict
from pynwb.file import Device

In [3]:
@register_class('OpticalChannel', 'ndx-multichannel-volume')
class OpticalChannel(NWBContainer):
    """An optical channel used to record from an imaging plane."""

    __nwbfields__ = ('description',
                     'emission_lambda')

    @docval({'name': 'name', 'type': str, 'doc': 'the name of this electrode'},  # required
            {'name': 'description', 'type': str, 'doc': 'Any notes or comments about the channel.'},  # required
            {'name': 'emission_lambda', 'type': float, 'doc': 'Emission wavelength for channel, in nm.'})  # required
    def __init__(self, **kwargs):
        description, emission_lambda = popargs("description", "emission_lambda", kwargs)
        super().__init__(**kwargs)
        self.description = description
        self.emission_lambda = emission_lambda


@register_class('ImagingVolume', 'ndx-multichannel-volume')
class ImagingVolume(NWBDataInterface):
    """An imaging plane and its metadata."""

    __nwbfields__ = ({'name': 'optical_channels', 'child': True},
                     'Order_optical_channels',
                     'description',
                     'device',
                     'location',
                     'conversion',
                     'origin_coords',
                     'origin_coords_units',
                     'grid_spacing',
                     'grid_spacing_units',
                     'reference_frame',
                     )

    @docval(*get_docval(NWBDataInterface.__init__, 'name'),  # required
            {'name': 'optical_channels', 'type': (list, OpticalChannel),  # required
             'doc': 'One of possibly many groups storing channel-specific data.'},
            {'name': 'Order_optical_channels', 'type':OpticalChannelReferences, 'doc':'Order of the optical channels in the data'},
            {'name': 'description', 'type': str, 'doc': 'Description of this ImagingVolume.'},  # required
            {'name': 'device', 'type': Device, 'doc': 'the device that was used to record'},  # required
            {'name': 'location', 'type': str, 'doc': 'Location of image plane.'},  # required
            {'name': 'reference_frame', 'type': str,
             'doc': 'Describes position and reference frame of manifold based on position of first element '
                    'in manifold.',
             'default': None},
            {'name': 'origin_coords', 'type': 'array_data',
             'doc': 'Physical location of the first element of the imaging plane (0, 0) for 2-D data or (0, 0, 0) for '
                    '3-D data. See also reference_frame for what the physical location is relative to (e.g., bregma).',
             'default': None},
            {'name': 'origin_coords_unit', 'type': str,
             'doc': "Measurement units for origin_coords. The default value is 'meters'.",
             'default': 'meters'},
            {'name': 'grid_spacing', 'type': 'array_data',
             'doc': "Space between pixels in (x, y) or voxels in (x, y, z) directions, in the specified unit. Assumes "
                    "imaging plane is a regular grid. See also reference_frame to interpret the grid.",
             'default': None},
            {'name': 'grid_spacing_unit', 'type': str,
             'doc': "Measurement units for grid_spacing. The default value is 'meters'.",
             'default': 'meters'})
    def __init__(self, **kwargs):
        keys_to_set = ('optical_channels',
                       'Order_optical_channels',
                       'description',
                       'device',
                       'location',
                       'reference_frame',
                       'origin_coords',
                       'origin_coords_unit',
                       'grid_spacing',
                       'grid_spacing_unit')
        args_to_set = popargs_to_dict(keys_to_set, kwargs)
        super().__init__(**kwargs)

        if not isinstance(args_to_set['optical_channels'], list):
            args_to_set['optical_channels'] = [args_to_set['optical_channels']]

        for key, val in args_to_set.items():
            setattr(self, key, val)


In [27]:
@register_class('VolumeSegmentation', 'ndx-multichannel-volume')
class VolumeSegmentation(DynamicTable):
    """
    Stores pixels in an image that represent different regions of interest (ROIs)
    or masks. All segmentation for a given imaging volume is stored together, with
    storage for multiple imaging planes (masks) supported. Each ROI is stored in its
    own subgroup, with the ROI group containing both a 3D mask and a list of pixels
    that make up this mask. Segments can also be used for masking neuropil. If segmentation
    is allowed to change with time, a new imaging plane (or module) is required and
    ROI names should remain consistent between them.
    """

    __fields__ = ('imaging_volume','name')

    __columns__ = (
        {'name': 'image_mask', 'description': 'Image masks for each ROI'},
        {'name': 'voxel_mask', 'description': 'Voxel masks for each ROI', 'index': True}
    )

    @docval({'name': 'description', 'type': str,  # required
             'doc': 'Description of image plane, recording wavelength, depth, etc.'},
            {'name': 'imaging_volume', 'type': ImagingVolume,  # required
             'doc': 'the ImagingVolume this ROI applies to'},
            {'name': 'name', 'type': str, 'doc': 'name of VolumeSegmentation.', 'default': None},
            *get_docval(DynamicTable.__init__, 'id', 'columns', 'colnames'))
    def __init__(self, **kwargs):
        imaging_volume = popargs('imaging_volume', kwargs)
        if kwargs['name'] is None:
            kwargs['name'] = imaging_volume.name
        super().__init__(**kwargs)
        self.imaging_volume = imaging_volume

    @docval({'name': 'voxel_mask', 'type': 'array_data', 'default': None,
             'doc': 'voxel mask for 3D ROIs: [(x1, y1, z1, weight1, ID), (x2, y2, z2, weight2, ID), ...]',
             'shape': (None, 5)},
            {'name': 'image_mask', 'type': 'array_data', 'default': None,
             'doc': 'image with the same size of image where positive values mark this ROI',
             'shape': [[None]*3]},
            {'name': 'id', 'type': int, 'doc': 'the ID for the ROI', 'default': None},
            allow_extra=True)
    def add_roi(self, **kwargs):
        """Add a Region Of Interest (ROI) data to this"""
        voxel_mask, image_mask = popargs('voxel_mask', 'image_mask', kwargs)
        if image_mask is None and voxel_mask is None:
            raise ValueError("Must provide 'image_mask' and/or 'voxel_mask'")
        rkwargs = dict(kwargs)
        if image_mask is not None:
            rkwargs['image_mask'] = image_mask
        if voxel_mask is not None:
            rkwargs['voxel_mask'] = voxel_mask
        return super().add_row(**rkwargs)

    @staticmethod
    def voxel_to_image(voxel_mask):
        """Converts a #D pixel_mask of a ROI into an image_mask."""
        image_matrix = np.zeros(np.shape(voxel_mask))
        npmask = np.asarray(voxel_mask)
        x_coords = npmask[:, 0].astype(np.int32)
        y_coords = npmask[:, 1].astype(np.int32)
        z_coords = npmask[:, 2].astype(np.int32)
        weights = npmask[:, -1]
        image_matrix[y_coords, x_coords, z_coords] = weights
        return image_matrix

    @staticmethod
    def image_to_pixel(image_mask):
        """Converts an image_mask of a ROI into a pixel_mask"""
        voxel_mask = []
        it = np.nditer(image_mask, flags=['multi_index'])
        while not it.finished:
            weight = it[0][()]
            if weight > 0:
                x = it.multi_index[0]
                y = it.multi_index[1]
                z = it.multi_index[2]
                voxel_mask.append([x, y, z, weight])
            it.iternext()
        return voxel_mask

    @docval({'name': 'description', 'type': str, 'doc': 'a brief description of what the region is'},
            {'name': 'region', 'type': (slice, list, tuple), 'doc': 'the indices of the table', 'default': slice(None)},
            {'name': 'name', 'type': str, 'doc': 'the name of the ROITableRegion', 'default': 'rois'})
    def create_roi_table_region(self, **kwargs):
        return self.create_region(**kwargs)
    


In [12]:
@register_class('MultiChannelVolume', 'ndx-multichannel-volume')
class MultiChannelVolume(NWBDataInterface):
    """An imaging plane and its metadata."""

    __nwbfields__ = ('resolution',
                     'description',
                     'channels',
                     'data',
                     'imaging_volume'
                     )

    @docval(*get_docval(NWBDataInterface.__init__, 'name'),  # required
            {'name': 'resolution', 'type': list, 'doc':'pixel resolution of the image', 'shape':[None]},
            {'name': 'imaging_volume', 'type': ImagingVolume, 'doc': 'the Imaging Volume the data was generated from'},
            {'name': 'description', 'type': str, 'doc':'description of image'},
            {'name': 'channels', 'doc': 'description of what each channel in the image maps to', 'type': list, 'shape':[None]},
            {'name': 'data', 'doc': 'Volumetric multichannel data', 'type': 'array_data', 'shape':[None]*4},
    )
    
    def __init__(self, **kwargs):
        keys_to_set = ('resolution',
                       'description',
                       'channels',
                       'data',
                       'imaging_volume'
                       )
        args_to_set = popargs_to_dict(keys_to_set, kwargs)
        super().__init__(**kwargs)

        for key, val in args_to_set.items():
            setattr(self, key, val)

### Creating NWB file for NeuroPAL data

In [13]:
import numpy as np
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.behavior import SpatialSeries, Position
from pynwb.image import ImageSeries
from pynwb.ophys import OnePhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, CorrectedImageStack, MotionCorrection, RoiResponseSeries
from datetime import datetime
from dateutil import tz
import pandas as pd

In [36]:
'''
NWB files should include
General description of experimental conditions
Time Series GCAMP data if available
Multichannel NeuroPAL volume (extension)
Point cloud of neuron centers (nx3)
Associated Metadata

'''

session_start_time = datetime(2022,2,12, tzinfo=tz.gettz("US/Pacific"))

print(session_start_time)

nwbfile = NWBFile(
    session_description = "Worm head",
    identifier = "2022-02-12-w01-NP1",
    session_start_time = session_start_time,
    lab = "FOCO lab",
    institution = "UCSF",
    related_publications = ""
)

In [37]:
nwbfile.subject = Subject(
    subject_id= "w01",
    age = "YA",
    description = "worm 1",
    species = "C elegan",
    sex = "XO"
)

In [38]:
'''
Define Imaging plane which will include optical_channel and devic data
'''

device = nwbfile.create_device(
    name="Microscope",
    description="One-photon microscope: Weill",
    manufacturer="Leica"
)

channel_1 = OpticalChannel(
    name="mNeptune 2.5",
    description="561-700-75m",
    emission_lambda=561.
)

channel_2 = OpticalChannel(
    name="Tag RFP-T",
    description="561-605-70m",
    emission_lambda=561.
)
    
channel_3 = OpticalChannel(
    name="CyOFP1",
    description="488-605-70m",
    emission_lambda=488.
)

channel_4 = OpticalChannel(
    name="GFP-GCaMP",
    description="488-525-50m",
    emission_lambda=488.
)

channel_5 = OpticalChannel(
    name="mTagBFP2",
    description="405-460-50m",
    emission_lambda=405.
)
    
channel_6 = OpticalChannel(
    name="mNeptune 2.5 - high excite",
    description="639-700-75m",
    emission_lambda=639.
)

OpticalChannelRefs = OpticalChannelReferences(
    name = 'OpticalChannelRefs',
    data = [channel_1.description, channel_2.description, channel_3.description, channel_4.description, channel_5.description, channel_6.description]
)

imaging_vol = ImagingVolume(
    name= "imaging_volume",
    optical_channels=[channel_1, channel_2, channel_3, channel_4, channel_5, channel_6],
    Order_optical_channels = OpticalChannelRefs,
    description="NeuroPAL image of C elegan brain",
    device = device,
    location = "head",
    grid_spacing = [0.3208, 0.3208, 0.75],
    grid_spacing_unit = 'micrometers',
    origin_coords=[0,0,0],
    origin_coords_unit = 'micrometers',
    reference_frame = 'Head of C elegan with [0,0,0] in top left of the image and oriented as posterior on the left and ventral on top.'
)

In [39]:
'''
Create plane_segmentation to define object that you add ROIs to
Add voxel masks to display ROIs using arrays of triplets (x,y,weight)
For our purposes just use weight of 1
Can add time series fluorescence measurements for these ROIs if you want to 
''' 

vs = VolumeSegmentation(
    name = 'VolumeSegmentation',
    description = 'Neuron centers for multichannel volumetric image',
    imaging_volume = imaging_vol
)


csv = pd.read_csv('data/NP_FOCO_cropped/2022-02-12-w01-NP1/blobs_og.csv') # read ROIs from blobs.csv

voxel_mask = []

for i, row in csv.iterrows():
    x = row['X']
    y = row['Y']
    z = row['Z']
    ID = row['ID']

    voxel_mask.append((x,y,z,1,ID))

vs.add_roi(voxel_mask=voxel_mask)

### Adding multichannel volume

In [40]:
import skimage.io as skio

data = np.transpose(skio.imread('data/NP_FOCO_cropped/2022-02-12-w01-NP1/neuropal_1_MMStack_Pos0.ome.tif'))


In [41]:
image =  MultiChannelVolume(
    name = '2022-02-12-w01-NP1',
    imaging_volume = imaging_vol,
    resolution = [0.3208, 0.3208, 0.75],
    description = '2022-02-12-w01-NP1',
    channels = ['red', 'white', 'green', 'GCAMP', 'blue', 'extra-red'],
    data = data
)

### Load everything into NWB file and write

In [42]:
'''
Writing NWB file
'''

nwbfile.add_acquisition(image)

neuroPAL_module = nwbfile.create_processing_module(
    name= 'neuroPAL',
    description = 'neuroPAL image data and metadata'
)

neuroPAL_module.add(vs)
neuroPAL_module.add(imaging_vol)
neuroPAL_module.add(OpticalChannelRefs)

io = NWBHDF5IO("Example_NWB.nwb", mode="w")
io.write(nwbfile)
io.close()

writing


/Users/danielsprague/anaconda3/envs/nwb/lib/python3.8/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'MultiChannelVolume/data': Value with data type uint16 is being converted to data type int16 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Users/danielsprague/anaconda3/envs/nwb/lib/python3.8/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'ImagingVolume/origin_coords': Value with data type int64 is being converted to data type float64 (min specification: float32).
  warnings.warn(full_warning_msg, DtypeConversionWarning)


TypeError: Can't implicitly convert non-string objects to strings

In [35]:
io.close()

In [16]:
print(channel_1.description)

561-700-75m


### Adding GCAMP data

In [ ]:
'''
Create time series object for GCAMP time series data
TODO: update to be OptPhys GCAMP time series
Can also add time series for stimulus data if necessary
'''

data = load('datapath') #first dimension must be time, second and third dimensions represent X and Y, optional fourth dimension is Z

GCAMP_time_series = OnePhotonSeries(
    name = "GCAMP",
    data= data,
    rate=1.0,
    unit='normalized amplitude'
)

nwbfile.add_acquisition(GCAMP_time_series)